In [ ]:
import os
import shutil
import subprocess
from pathlib import Path

def get_dir_size(path):
    total = 0
    try:
        for entry in os.scandir(path):
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    except (PermissionError, FileNotFoundError):
        pass
    return total

def format_size(size):
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size < 1024:
            return f"{size:.2f}{unit}"
        size /= 1024
    return f"{size:.2f}TB"

workspace = Path("/workspaces/Allentown-L104-Node")

print("🚀 Starting Data Space Fix")

# 1. Analyze
print("\n📊 Analyzing large directories...")
targets = [
    workspace / "node_modules",
    workspace / ".venv",
    workspace / ".venv-1",
    workspace / "__pycache__",
    workspace / ".git",
    workspace / "data",
    workspace / "archive",
    workspace / ".l104_backups"
]

for target in targets:
    if target.exists():
        size = get_dir_size(target)
        print(f"   {target.name}: {format_size(size)}")

# 2. Cleanup __pycache__
print("\n🧹 Cleaning up __pycache__ files...")
count = 0
for p in workspace.rglob("__pycache__"):
    if p.is_dir():
        shutil.rmtree(p)
        count += 1
print(f"   Removed {count} __pycache__ directories.")

# 3. Git GC
print("\n⚙️ Running Git Garbage Collection...")
try:
    subprocess.run(['git', 'gc', '--aggressive', '--prune=now'], cwd=workspace, capture_output=True)
    print("   Git GC complete.")
except Exception as e:
    print(f"   Git GC failed: {e}")

# 4. Cleanup external caches
print("\n🧹 Cleaning up external caches...")
for cache_path in [Path.home() / ".npm", Path.home() / ".cache/pip"]:
    if cache_path.exists():
        print(f"   Removing {cache_path}...")
        shutil.rmtree(cache_path, ignore_errors=True)

print("\n✅ Data space optimization complete.")


In [ ]:
import os
print(os.getcwd())
print("Terminal-like test")
